このノートブックは直接的な解放には役に立たない可能性があるが,  
現在の文章がどのようなトピックによって構成されているかをLDAを用いて調べてみる。

参考 [Python Machine Learning (3rd Ed.) Code Repository](https://github.com/rasbt/python-machine-learning-book-3rd-edition/blob/master/ch08/ch08.py)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch

In [ ]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
valid_df = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")

In [ ]:
more_valid=pd.DataFrame(valid_df["more_toxic"])
less_valid=pd.DataFrame(valid_df["less_toxic"])

In [ ]:
more_un_valid=more_valid["more_toxic"].unique()
less_un_valid=less_valid["less_toxic"].unique()
print("more",len(more_valid["more_toxic"].unique()),len(more_valid))
print("less",len(less_valid["less_toxic"].unique()),len(less_valid))

In [ ]:
all_valid=pd.DataFrame(more_valid["more_toxic"].unique().tolist()+less_valid["less_toxic"].unique().tolist())

In [ ]:
less_un_valid[1]

In [ ]:
all_valid.iloc[11678+1]

ここで,BoWを作成する.
[公式document](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words="english",
                       max_df=.1,
                       max_features=500)
X=count.fit_transform(all_valid[0].values)

In [ ]:
tmp=10
X[tmp].toarray(),all_valid.iloc[tmp]

ここでLDAを作成する. [LatentDirichletAllocation](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as lda
lda_valid=lda(n_components=10,random_state=42,learning_method="batch")
X_topics=lda_valid.fit_transform(X)

In [ ]:
lda_valid.components_.shape

In [ ]:
n_top_words=5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda_valid.components_):
    print("Topic %d:" % (topic_idx+1))
    print(" ".join([feature_names[i] for i in topic.argsort() \
                   [:-n_top_words-1:-1]]))

酷い形のトピックが抽出されていることがわかります.

また,今回は相対的に
toxicかそうでないかを見ているため,単語ごとにどちらの郡に属しているかは参考程度にしかなりません.

In [ ]:
for topic_id in range(10):
    tmp=X_topics[:,topic_id].argsort()[::-1]
    for iter_idx, movie_idx in enumerate(tmp[:3]):
        print(f"\n Topic{topic_id+1} #%d" % (iter_idx+1))
        
        if movie_idx < len(more_valid["more_toxic"].unique()):
            print("more toxic",movie_idx)

        else:
            print("less toxic",movie_idx)

        print(all_valid.iloc[movie_idx][:100])

結論 : あまり参考にならなかった.  